In [ ]:
import pandas as pd
from scipy.stats import gmean

In [ ]:
import plotly.graph_objects as go

In [ ]:
filename_DD = f"../data/df_DD_for_SARIMAX_2025-04-08_14-28-37.csv"
filename_FB = f"../data/df_FB_for_SARIMAX_2025-04-08_14-28-37.csv"
df_DD = pd.read_csv(filename_DD, index_col=None, parse_dates=["datetime_hour"])
df_FB = pd.read_csv(filename_FB, index_col=None, parse_dates=["datetime_hour"])

In [ ]:
cols_to_keep = ["hex_id", 'datetime_hour', 'rent_count', 'return_count', 'is_dayoff']

In [ ]:
df_DD = df_DD[cols_to_keep]

In [ ]:
df_FB = df_FB[cols_to_keep]

In [ ]:
df_FB["city"]="Freiburg"
df_DD["city"]="Dresden"

In [ ]:
df_bike = pd.concat([df_DD, df_FB], axis=0, ignore_index=True)

In [ ]:
outlier_days = ["2024-03-22",  "2024-03-01", "2024-03-02", "2024-02-02", "2023-06-24"]
outlier_days = [pd.to_datetime(day).date() for day in outlier_days]
outlier_days

In [ ]:
flt = df_bike.datetime_hour.dt.date.isin(outlier_days)
df_bike = df_bike[~flt]

In [ ]:
filename_events = "../data/events/df_events_with_hex_id_and_all_cols_2025-04-23_13-12-48.csv"
df_events = pd.read_csv(filename_events, index_col=None, parse_dates=["StartDateTime", "EndDateTime"])

In [ ]:
flt = df_events.StartDateTime.dt.date.isin(outlier_days)
df_events = df_events[~flt]

In [ ]:
flt = df_events.EndDateTime < df_events.StartDateTime
#  replace date of EndDateTime with next day but keep time

df_events.loc[flt, "EndDateTime"] = df_events.loc[flt, "EndDateTime"] + pd.DateOffset(days=1)

In [ ]:
flt = df_events.EndDateTime < df_events.StartDateTime
assert len(df_events[flt])==0

In [ ]:
flt = df_events.EndDateTime.isna()
df_events.loc[flt, "EndDateTime"] = df_events.loc[flt, "StartDateTime"] + pd.DateOffset(hours=3)

In [ ]:
df_events.StartDateTime = df_events.StartDateTime.dt.floor("h")
df_events.EndDateTime = df_events.EndDateTime.dt.floor("h")

In [ ]:
hex_id_grouping = df_bike.groupby("hex_id")["rent_count"].sum()

In [ ]:
hex_id_grouping = hex_id_grouping[hex_id_grouping>5000]

In [ ]:
allowed_hex_ids = hex_id_grouping.index.tolist()

In [ ]:
allowed_hex_ids

In [ ]:
df_events = df_events[df_events.hex_id.isin(allowed_hex_ids)].copy()

In [ ]:
df_events.Source.value_counts()

In [ ]:
football_events = df_events.loc[df_events.Source.isin(["dynamo-dresden.de", "scfreiburg.com"])]
football_events

In [ ]:
df_events.EventCategory.fillna("Unknown", inplace=True)

In [ ]:
df_events["category"] = df_events.SourceGroup + " - " + df_events.EventCategory

In [ ]:
df_events["category"].value_counts(dropna=False)

# graph some events - which hours to take?

In [ ]:
flt_tmp = (df_events.EndDateTime.dt.hour!=0) & (df_events.StartDateTime.dt.date == df_events.EndDateTime.dt.date)
df_tmp = df_events.loc[flt_tmp]

In [ ]:
pd.to_datetime("2024-09-29").date()

In [ ]:
football_events.StartDateTime.dt.date.iloc[0]

In [ ]:
football_events.StartDateTime.dt.date == pd.to_datetime("2024-09-29").date()

In [ ]:
# df_tmp = df_tmp.sample((10))
df_tmp = football_events

In [ ]:
df_tmp

In [ ]:
# for return,take this hour and previous hour
TIGHT_MARGIN=10

for rent_or_return in ["rent", "return"]:
    for i, row in df_tmp.iloc[[2]].iterrows():
        hex_id = row.hex_id
        start_hour = row.StartDateTime.hour
        end_hour = row.EndDateTime.hour
        date = row.StartDateTime.date()
        event_day = row.StartDateTime.date()

        day_last_week = (date - pd.DateOffset(weeks=1)).date()
        day_next_week = (date + pd.DateOffset(weeks=1)).date()

        fig = go.Figure()

        label_names= ["event_day", "day_last_week", "day_next_week"] 
        color_dict = {
            "event_day": "orange",
            "day_last_week": "blue",
            "day_next_week": "cornflowerblue"
        }

        linestyle_dict = {
            "event_day": "solid",
            "day_last_week": "dash",
            "day_next_week": "dash"
        }

        for label, dt in zip(label_names , [date, day_last_week, day_next_week]):
            flt = (df_bike.datetime_hour.dt.date == dt) & (df_bike.hex_id == hex_id)
            line_df = df_bike.loc[flt, ["datetime_hour", "rent_count", "return_count"]].copy()
            if len(line_df) > 0:
                line_df['hour'] = line_df.datetime_hour.dt.hour
                colname = "rent_count" if rent_or_return == "rent" else "return_count"

                fig.add_trace(go.Scatter(x = line_df.hour, 
                                            y = line_df[colname],name=label, 
                                            line=dict(color=color_dict[label],
                                                        dash=linestyle_dict[label],)))
                
                
                
                max_value = line_df[colname].max()
        
        # annotate start_hour and end_hour with vertical lines
        if rent_or_return == "return":
            fig.add_trace(go.Scatter(x=[start_hour, start_hour], y=[0, 35], mode='lines', name='Event start hour', line=dict(color='red', width=2)))
        else:
            fig.add_trace(go.Scatter(x=[end_hour, end_hour], y=[0, 35], mode='lines', name='Event end hour', line=dict(color='pink', width=2)))


        fig.update_layout(
            xaxis_title="Hour",
            yaxis_title={"rent": "Rent count", "return": "Return count"}[rent_or_return],
            xaxis=dict(tickmode='linear', dtick=1),
            yaxis=dict(tickmode='linear', dtick=5,range=[0,35]),
            width=700, height=400,
            margin=dict(l=TIGHT_MARGIN, r=TIGHT_MARGIN, t=TIGHT_MARGIN, b=TIGHT_MARGIN)
            # title=str(event_day) "2024-01-28"
        )


        fig.show()

    

In [ ]:
# CONCLUSION 
# for rent, take the end hour and the hout after that

# for return,take stat hour and previous hour

# compute effect for each event individually

In [ ]:
# flt1 = (df_events.StartDateTime.dt.date == df_events.EndDateTime.dt.date).value_counts()
# flt1

In [ ]:
# flt2 = df_events.EndDateTime.isna().value_counts()
# flt2

In [ ]:
df_bike.columns

In [ ]:
variables= ["rent_count", "return_count"]
labels = ["event", "last_week", "next_week"]
hour_categories_helper = {
    "rent_count": ["end_hour", "end_hour_plus_1"],
    "return_count" : ["start_hour", "start_hour_minus_1", "start_hour_minus_2"]
}

In [ ]:
# for rent, take the end hour and the hout after that (maybe only end hour)

# for return,take stat hour and previous hour (ad maybe 2 h before )

event_stats = []

for i, row in df_events.iterrows():
    category = row.category
    hex_id = row.hex_id
    start_hour = row.StartDateTime.hour

    start_datetime = row.StartDateTime
    end_datetime = row.EndDateTime
    if pd.isna(end_datetime):
        end_datetime = start_datetime + pd.DateOffset(hours=3)

    end_hour = end_datetime.hour
    event_day = row.StartDateTime.date()

    flt_hex_id = (df_bike.hex_id == hex_id)

    hour_helper = {
        "return_count": {"event": start_datetime,
                        "last_week": start_datetime-pd.DateOffset(weeks=1),
                        "next_week": start_datetime+pd.DateOffset(weeks=1)
                        },
        "rent_count": {
            "event": end_datetime, 
            "last_week": end_datetime-pd.DateOffset(weeks=1), 
            "next_week": end_datetime+pd.DateOffset(weeks=1) 
        }
    }

    hour_offset_helper = {
        "end_hour": pd.DateOffset(hours=0),
        "end_hour_plus_1": pd.DateOffset(hours=1),
        "start_hour": pd.DateOffset(hours=0),
        "start_hour_minus_1": pd.DateOffset(hours=-1),
        "start_hour_minus_2": pd.DateOffset(hours=-2)
    }

    
    for variable in variables:
        for label in labels:
            for hour_category in hour_categories_helper[variable]:
                base_hour = hour_helper[variable][label]
                specific_hour = base_hour + hour_offset_helper[hour_category]
                if hour_category == "start_time":
                    assert base_hour==specific_hour

                flt_date = df_bike.datetime_hour == specific_hour
                flt = flt_hex_id & flt_date
                slice_tmp = df_bike.loc[flt, variable].copy()
                if not slice_tmp.empty:
                    print(f"{base_hour} data for {variable} found")
                    mean = slice_tmp.mean()
                    df_events.at[i, f"{variable}_{label}_{hour_category}"]=mean
                else:
                    print(f"{base_hour} data for {variable} not found")

# df_slice

## find out what hours are needed

In [ ]:
df_slice = df_events.copy()

In [ ]:
df_slice.drop(columns = ["_merge", "lat", "lon", "PLZ_GEMA", 
                         "Datum_Nutzungsbeginn_GEMA", "Datum_Nutzungsende_GEMA", "Ort_GEMA", "Strasse_GEMA",
                         "Hausnummer_GEMA", "EndDate", "EndTime", "HouseNumber", "Street_orig", "StreetHouseNumber_orig", "Street", "StreetHouseNumber"], errors="ignore", inplace=True)

In [ ]:
df_slice

In [ ]:
flt = df_slice.category == "deecooob GmbH - Unknown"
df_slice.loc[flt, "category"] = "deecoob GmbH - " + df_slice.loc[flt, "Source"]


In [ ]:
df_slice.category.value_counts()

In [ ]:
df_slice.columns

In [ ]:
added_columns = [col for col in df_slice.columns if col.startswith("rent_count_") or col.startswith("return_count_")]

In [ ]:
added_columns

In [ ]:
for col in added_columns:
    flt = (df_slice[col] < 10)

    df_slice.loc[flt, col] = pd.NA
    
# df_slice[((df_slice.rent_count_end_hour_other_weeks >= 10) & (df_slice.rent_count_event_end_hour >= 10)) |  ((df_slice.return_count_event_start_hour >= 10) & (df_slice.return_count_other_weeks_start_hour >= 10 ))]

In [ ]:
hour_categories_helper

In [ ]:
labels

In [ ]:
hour_categories = hour_categories_helper["rent_count"] + hour_categories_helper["return_count"]

In [ ]:
hour_categories

In [ ]:
labels

In [ ]:
for variable in variables:
     for hour_category in hour_categories_helper[variable]:
          # colname is f"{variable}_{label}_{hour_category}"
          df_slice[f'{variable}_other_weeks_{hour_category}'] = df_slice[[f"{variable}_last_week_{hour_category}", f"{variable}_next_week_{hour_category}"]].mean(axis=1, skipna=True)

In [ ]:
for variable in variables:
     for hour_category in hour_categories_helper[variable]:
          df_slice.drop(columns=f"{variable}_last_week_{hour_category}", errors="ignore", inplace=True)
          df_slice.drop(columns=f"{variable}_next_week_{hour_category}", errors="ignore", inplace=True)



In [ ]:
df_slice.columns

In [ ]:
hour_categories_helper

In [ ]:
for variable in variables:
    for hour_category in hour_categories_helper[variable]:
        df_slice[f'{variable}_{hour_category}_ratio'] =df_slice[f"{variable}_event_{hour_category}"] /df_slice[f"{variable}_other_weeks_{hour_category}"]

In [ ]:
ratio_columns = [col for col in df_slice.columns if '_ratio' in col]
ratio_columns

In [ ]:
df_slice

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
df_slice[ratio_columns].describe()

In [ ]:
df_slice[ratio_columns].boxplot()
plt.xticks(rotation=90)
plt.ylim(-1, 5)
plt.show()


In [ ]:
import numpy as np

for col in ratio_columns:
    df_slice[col] = df_slice[col].replace(0, pd.NA)
    df_slice[col] = df_slice[col].replace(np.inf, pd.NA)

In [ ]:
gmean([0.1,2], nan_policy="omit")

In [ ]:
def gmean_with_nan_handling(series):
	# Convert the series to numeric, coercing errors to NaN
	numeric_series = pd.to_numeric(series, errors='coerce')
	return gmean(numeric_series.dropna(), nan_policy="omit")

def count_non_na(series):
	# Count the number of non-NA/null observations
	return len(series.dropna())

# Apply the custom function
#stats_by_category = df_slice.groupby("category")[ratio_columns].agg([gmean_with_nan_handling, count_non_na])

In [ ]:
stats_by_category = df_slice.groupby("category")[ratio_columns].agg(["mean"])

In [ ]:
df_slice.loc[df_slice.category=="GEMA - Nicht geographisch", ratio_columns]

In [ ]:
stats_by_category.columns

In [ ]:
stats_by_category

In [ ]:
# CONCLUSION: for end, take only end_hour. For start, take all three hours. 



In [ ]:
stats_by_category = stats_by_category.astype(float)

In [ ]:
stats_by_category.describe()

In [ ]:
stats_by_category.boxplot()
plt.xticks(rotation=90)
plt.ylim(-1, 5)
plt.show()

In [ ]:
stats_by_category = df_slice.groupby("category")[ratio_columns].agg(["mean", count_non_na])

In [ ]:
stats_by_category

In [ ]:
# df_events.sort_values("rent_ratio", ascending=False)

## group hours

In [ ]:
df_slice.columns

In [ ]:
df_slice.drop(columns = ratio_columns, inplace=True, errors="ignore") 
df_slice.drop(columns = [col for col in df_slice.columns if col.endswith("_end_hour_plus_1")], inplace=True, errors="ignore") 


In [ ]:
for v in ["other_weeks", "event"]:
    cols_for_avg = [f'return_count_{v}_start_hour', f'return_count_{v}_start_hour_minus_1', f'return_count_{v}_start_hour_minus_2']
    df_slice[f"return_count_{v}_start_avg"] = df_slice[cols_for_avg].mean(axis=1)
    df_slice.drop(columns = cols_for_avg, errors='ignore', inplace=True)


In [ ]:
df_slice["rent_count_ratio"] = df_slice.rent_count_event_end_hour / df_slice.rent_count_other_weeks_end_hour
df_slice["return_count_ratio"] = df_slice.return_count_event_start_avg / df_slice.return_count_other_weeks_start_avg


In [ ]:
df_slice

In [ ]:
new_ratio_cols = [col for col in df_slice.columns if col.endswith("_ratio")]

In [ ]:
flt= df_slice[new_ratio_cols].isna().all(axis=1)
df_slice = df_slice[~flt]

In [ ]:
len(df_slice)

In [ ]:
stats_by_category = df_slice.groupby("category")[new_ratio_cols].agg(["mean", count_non_na])

In [ ]:
stats_by_category

In [ ]:
df_slice["major_category"] =df_slice.category

In [ ]:
df_slice.major_category.value_counts().sort_index()

In [ ]:
df_slice.major_category = df_slice.major_category.replace(to_replace=["deecoob GmbH - eventim.de",
                                            "deecoob GmbH - bandsintown.com",
                                            "deecoob GmbH - jambase.com",
                                             "deecoob GmbH - setlist.fm",
                                            "deecoob GmbH - concertful.com", "deecoob GmbH - eventbrite.com",
                                            "deecoob GmbH - songkick.com",	"deecoob GmbH - ticketmaster.de", "deecoob GmbH - wegow.com"],
                                value="deecoob GmbH - Music events")

df_slice.major_category = df_slice.major_category.replace(to_replace="deecoob GmbH - facebook.com",
                                value="deecoob GmbH - Facebook events")



df_slice.major_category = df_slice.major_category.replace("Self scraped - Feste, Märkte & Messen", "Self scraped - Festivals, fairs and markets")

df_slice.major_category = df_slice.major_category.replace("Self scraped - Ausstellungen & Museen", "Self scraped - Exhibitions & Museums")

df_slice.major_category = df_slice.major_category.replace("GEMA - Freizeit/Sport/Kultur", "GEMA - Leisure, sport and culture")
df_slice.major_category = df_slice.major_category.replace("GEMA - Handel und Dienstleistungen", "GEMA - Trade and services")

df_slice.major_category = df_slice.major_category.replace(to_replace=["Self scraped - Unknown", 
                                                                      "Self scraped - Verschiedenes",
                                                                       "Self scraped - Konzerte",
                                                                       "Self scraped - Musik & Party",
                                                                      "Self scraped - Kultur", "Self scraped - Theater & Schauspiel",
                                                                       "Self scraped - Verschiedenes ", "Self scraped - Kabarett & Comedy", 
                                                                       "Self scraped - Burgen & Schlösser"],
                                value="Self scraped - Other")


df_slice.major_category = df_slice.major_category.replace(to_replace=["GEMA - Kirchen",
                                                                      "GEMA - Gesundheitswesen (Ärzte)",
                                                                      "GEMA - Gesundheitswesen (Krankenhäuser)" ,
                                                                      "GEMA - Unknown"  ,
                                                                      "GEMA - Sozialeinrichtungen"   ,
                                                                      "GEMA - Staat und Verwaltung" ,
                                                                      "GEMA - Nicht geographisch",
                                                                      "GEMA - Bildungseinrichtungen"
                                                                      ],
                                value="GEMA - Other")



df_slice.major_category = df_slice.major_category.replace(to_replace=[    "GEMA - Gastronomie (mit Übernachtung)",
                                                                        "GEMA - Gastronomie (ohne Übernachtung)",
                                                                      ],
                                value="GEMA - Restaurants & Hotels")


df_slice.major_category = df_slice.major_category.replace(to_replace=[    "Self scraped - Football match",
                                                                        "Self scraped - Sportveranstaltungen",
                                                                      ],
                                value="Self scraped - Sports events")



In [ ]:
df_slice.major_category.value_counts().sort_index()

In [ ]:
df_slice

In [ ]:
import plotly

## check hypothesiis with multiple days

In [ ]:
df_slice[(df_slice.isOnMultipleDays==True) & (df_slice.EventCategory!="Ausstellungen & Museen")]

In [ ]:
df_slice["isOnMultipleDays_adj"]=False

In [ ]:
df_slice.Duration = pd.to_timedelta(df_slice.Duration)

In [ ]:
df_slice.Duration.dt.days.value_counts()

In [ ]:
df_slice.loc[df_slice.Duration.dt.days>=2, "isOnMultipleDays_adj"]=True

In [ ]:
df_slice.groupby(["isOnMultipleDays_adj"])[new_ratio_cols].agg(["mean", count_non_na])

In [ ]:
df_slice.groupby(["isOnMultipleDays"])[new_ratio_cols].agg(["mean", count_non_na])

In [ ]:
df_slice[df_slice.isOnMultipleDays==True]

In [ ]:
stats_by_major_category = df_slice.query("isOnMultipleDays==True").groupby("major_category")[new_ratio_cols].agg(["mean", count_non_na])
stats_by_major_category

In [ ]:
df_slice.query("isOnMultipleDays").groupby("major_category")[new_ratio_cols].agg(["mean", count_non_na])

## make plots

In [ ]:
final_stats = df_slice.groupby("major_category")[new_ratio_cols].agg(["mean", count_non_na])

In [ ]:
final_stats.reset_index(inplace=True)

In [ ]:
final_stats

In [ ]:
final_stats.columns = ["Event category", "Effect on demand", "Count events for demand effect", "Effect on supply", "Count events for supply effect"]

In [ ]:
final_stats["Effect on demand"] = final_stats["Effect on demand"]-1
final_stats["Effect on supply"] = final_stats["Effect on supply"]-1


In [ ]:
import plotly.express as px

In [ ]:
final_stats = final_stats.sort_values("Event category")

In [ ]:
final_stats

In [ ]:
fig = px.bar(final_stats, y = "Event category", x="Effect on demand", 
             width=800, height=400, color_discrete_sequence=["darkblue"]
             )
fig.update_layout(
    template="plotly_white",
    showlegend=False,  # Hide legend if color is fixed
    xaxis_tickformat=".0%",  
    xaxis = dict(range=[-1, 2.7], showline=True, linewidth=1, linecolor='black', mirror=True),
    yaxis=dict(showline=True, linewidth=1, linecolor='black', mirror=True)
)

labels = final_stats.apply(
    lambda row: f"{row['Effect on demand']:.0%} ({int(row['Count events for demand effect'])})", axis=1
)

fig.update_traces(
    text=labels,  # Format as percent
    textposition="outside"
)
fig.show()


In [ ]:
fig = px.bar(final_stats, y = "Event category", x="Effect on supply", 
             width=800, height=400, color_discrete_sequence=["darkblue"]
             )
fig.update_layout(
    template="plotly_white",
    showlegend=False,  # Hide legend if color is fixed
    xaxis_tickformat=".0%",  
    xaxis = dict(range=[-0.5, 1], showline=True, linewidth=1, linecolor='black', mirror=True),
    yaxis=dict(showline=True, linewidth=1, linecolor='black', mirror=True)
)

labels = final_stats.apply(
    lambda row: f"{row['Effect on supply']:.0%} ({int(row['Count events for supply effect'])})", axis=1
)

fig.update_traces(
    text=labels,  # Format as percent
    textposition="outside"
)
fig.show()
